In [19]:
import cv2
import math
import cvzone
from ultralytics import YOLO
import pandas as pd
import yaml
from collections import defaultdict
from sklearn.metrics import accuracy_score

In [20]:
waste_categories = {
    "Recyclable": {
        "Dry Waste": ["aluminum", "cardboard", "paper", "glass bottle", "plastic bag", "plastic bottle", "plastic container", "plastic straw"],
        "E-Waste": ["Keyboard", "Mobile Phone", "Monitor"]
    },
    "Non-Recyclable": {
        "Wet Waste": ["egg shell", "facemask", "food wrapper", "fruit peels", "treeleaves", "vegetable peels"]
    }
}

In [21]:
# Load YOLO model
model = YOLO("D:/Waste_Detection_New/best_0.pt")

In [22]:
# Load image
image_path = "D:\\Waste_Detection_New\\test\\images\\glass234_jpg.rf.9ce68d8fc0d788066e2975ebe99b027f.jpg"  
image = cv2.imread(image_path)

In [23]:
# Initialize waste summary dictionary
waste_summary = {"Recyclable": {"Dry Waste": defaultdict(int), "E-Waste": defaultdict(int)}, "Non-Recyclable": {"Wet Waste": defaultdict(int)}}

In [24]:
# Accuracy calculation variables
correct_predictions = 0
total_predictions = 0

In [25]:
# Object tracking setup
object_tracker = {}  # Track detected objects

In [26]:
# Run YOLOv8 detection on image
results = model(image)
predicted_labels = []


0: 640x640 2 glass bottles, 647.5ms
Speed: 4.0ms preprocess, 647.5ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)


In [27]:
for r in results:
    for box in r.boxes:
        class_id = int(box.cls[0])
        label = model.names[class_id]
        conf = box.conf[0].item()  # Confidence score
        predicted_labels.append(label)
        
        # Classify detected waste using static dictionary
        for category, subcategories in waste_categories.items():
            for subcategory, items in subcategories.items():
                if label in items:
                    waste_summary[category][subcategory][label] += 1
        
        # Draw bounding box and label on image
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image, f"{label} {conf:.2f}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
# Compare predictions with tracked objects
if len(object_tracker) == 0:
    object_tracker = {label: 1 for label in predicted_labels}  # First frame as reference
    
for label in predicted_labels:
    if label in object_tracker:
        correct_predictions += 1
    total_predictions += 1
    
# Calculate accuracy
image_accuracy = (correct_predictions / total_predictions) * 100 if total_predictions > 0 else 0

In [28]:
# Save accuracy to CSV
accuracy_data = pd.DataFrame({'Metric': ['Image Accuracy'], 'Value': [image_accuracy]})
accuracy_data.to_csv("image_model_accuracy.csv", index=False)

In [29]:
# Generate structured summary table
summary_list = []
for category, subcategories in waste_summary.items():
    for subcategory, items in subcategories.items():
        for item, count in items.items():
            summary_list.append([category, subcategory, item, count])

waste_df = pd.DataFrame(summary_list, columns=["Category", "Subcategory", "Waste Type", "Count"])

In [30]:
# Append accuracy to summary
accuracy_row = pd.DataFrame([["Model Performance", "", "Accuracy", image_accuracy]], columns=["Category", "Subcategory", "Waste Type", "Count"])
waste_df = pd.concat([waste_df, accuracy_row], ignore_index=True)
waste_df.to_csv("waste_summary.csv", index=False)

In [31]:
# Show detection image
cv2.imshow("Waste Detection", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [32]:
# Print final structured summary
print("Final Waste Classification Summary:")
print(waste_df)
print("Image Model Accuracy:", image_accuracy, "%")

Final Waste Classification Summary:
            Category Subcategory    Waste Type  Count
0         Recyclable   Dry Waste  glass bottle    2.0
1  Model Performance                  Accuracy  100.0
Image Model Accuracy: 100.0 %
